In [1]:
import os
import sys
home_dir = "../../"
module_path = os.path.abspath(os.path.join(home_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

from vespa.predict.logodds import *
import vespa.predict.utils as vespa_utils

/projects/ashehu/akabir4/venvs/hopper_vespa_marquet_from_source/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda:0


In [2]:
cache_dir=home_dir+"models/vespa_marquet/cache"
t5_condProbas = T5_condProbas(cache_dir=cache_dir)
model, tokenizer = t5_condProbas.prott5.get_model(1) #1=LOGODDS

In [11]:
seq = "AMVNSTHR" # eos token added
seq = " ".join(list(seq))
model.to("cpu")
input_ids = tokenizer(seq, return_tensors="pt").input_ids.to("cpu")
print(input_ids)
with torch.no_grad():  # only logits is useful for us
    logits = model(input_ids, labels=input_ids).logits
    logits = logits[0].detach().numpy() 

print(logits.shape)

tensor([[ 3, 19,  6, 17,  7, 11, 20,  8,  1]])
(9, 128)


In [14]:
tokenizer.convert_tokens_to_ids("▁V")

6

In [9]:
from pathlib import Path
seq_dict = vespa_utils.parse_fasta_input(Path(home_dir+"models/aa_common/datasets_pmd_analysis/pmd_sequences.fasta")) # this replaces the unknown amino acids UZO, not B
print(len(seq_dict))
protid_seq_dict_list = [(prot_id, seq) for prot_id, seq in  seq_dict.items()]
protid, seq = protid_seq_dict_list[0]
seq

11714


'MVNSTHRGMHTSLHLWNRSSYRLHSNASESLGKGYSDGGCYEQLFVSPEVFVTLGVISLLENILVIVAIAKNKNLHSPMYFFICSLAVADMLVSVSNGSETIVITLLNSTDTDAQSFTVNIDNVIDSVICSSLLASICSLLSIAVDRYFTIFYALQYHNIMTVKRVGIIISCIWAACTVSGILFIIYSDSSAVIICLITMFFTMLALMASLYVHMFLMARLHIKRIAVLPGTGAIRQGANMKGAITLTILIGVFVVCWAPFFLHLIFYISCPQNPYCVCFMSHFNLYLILIMCNSIIDPLIYALRSQELRKTFKEIICCYPLGGLCDLSSRY'

In [17]:
a_prot_seq = protid_seq_dict_list[0]
csv_dir = home_dir+"models/vespa_marquet/outputs/vespa/lm_outputs"
mutation_gen = vespa_utils.MutationGenerator(sequence_dict=a_prot_seq)
proba_dict = t5_condProbas.get_proba_dict(a_prot_seq, mutation_gen)
dmiss_data = t5_condProbas.get_log_odds(proba_dict)
T5_condProbas.write_csv_dir(dmiss_data, csv_dir, mutation_gen)

OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 14.76 GiB total capacity; 9.02 GiB already allocated; 17.75 MiB free; 9.02 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [17]:
import re
seq = "CUDA"
re.sub(r"[UZOB]", "X", seq)

'CXDA'

In [ ]:
import vespa.predict.utils as vespa_utils
from pathlib import Path
seq_dict = vespa_utils.parse_fasta_input(Path(home_dir+"models/aa_common/datasets_pmd_analysis/pmd_sequences.fasta"))
print(len(seq_dict))
# protid_seq_dict_list = [{prot_id: seq} for prot_id, seq in  seq_dict.items()]
protid_seq_dict_list = [(prot_id, seq) for prot_id, seq in  seq_dict.items()]
protid, seq = protid_seq_dict_list[0]
print(protid, seq)